In [1]:
# load raw data and cleaned data
import pandas as pd
import numpy as np
import re
import swifter
import warnings
warnings.filterwarnings("ignore")
import string

raw_data = pd.read_csv("../raw data/timber.csv")
cleaned_data = pd.read_csv("../cleaned data/timber.csv")

In [2]:
def count_diff(old, new, columns):
  old_df = old.copy().replace('', np.nan).fillna('redacted')
  new_df = new.copy().replace('', np.nan).fillna('redacted')
  correct = 0
  for col in columns:
    if col not in old_df.columns:
      continue
    for i in range(len(new_df[col])):
      if new_df[col][i] == old_df[col][i]:
        correct += 1
  return len(old_df)*len(columns) - correct

In [3]:
cleaned_data.columns

Index(['ID', 'Date', 'DeclarationNumber', 'CustomsCode', 'SenderTaxID',
       'SenderNameEng', 'SenderAddressEng', 'RecipientNameEng',
       'RecipientAddressEng', 'TradingCountryCode', 'DestinationCountryCode',
       'DestinationCountryEng', 'PortUnladingEng', 'ShipmentDescriptionEng',
       'HS', 'NetWeightKG', 'StatValue_USD', 'USD_per_KG',
       'TradingCountryName', 'DestinationCountryName', 'HTS4',
       'SenderCompanyKeywordsEng', 'RecipientCompanyKeywordsEng'],
      dtype='object')

In [4]:
# num_cols: columns that should be numerical -> check for incorrect format
num_cols = ['ID', 'Date', 'DeclarationNumber', 'CustomsCode', 'SenderTaxID', 'HS', 'NetWeightKG', 'StatValue_USD', 'USD_per_KG', 'HTS4']

# str_cols: columns that should be string -> check for misspellings
str_cols = list(cleaned_data.columns)
for col in num_cols:
  str_cols.remove(col)

In [5]:
print(f"In raw data:")
print(f"{len(list(raw_data.columns))} columns, {len(raw_data)} rows")
print(f"{100*raw_data.isna().sum().sum()/(len(raw_data)*len(raw_data.columns))} % of missing values.")
print(f"{100*count_diff(raw_data, cleaned_data, num_cols)/(len(raw_data)*len(num_cols))} % of incorrect formats.")
print(f"{100*count_diff(raw_data, cleaned_data, str_cols)/(len(raw_data)*len(str_cols))} % of misspellings.")
print(f"\n{100-100*count_diff(raw_data, cleaned_data, list(cleaned_data.columns))/(len(raw_data)*len(list(cleaned_data.columns)))} % of correct values.")

In raw data:
29 columns, 3087822 rows
3.189506255932789 % of missing values.
87.93393531103801 % of incorrect formats.
93.19468544495116 % of misspellings.

9.092597221967594 % of correct values.


In [6]:
print(f"In cleaned data:")
print(f"{len(cleaned_data.columns)} columns, {len(cleaned_data)} rows")
print(f"{100*cleaned_data.isna().sum().sum()/(len(cleaned_data)*len(cleaned_data.columns))} % of missing values.")

In cleaned data:
23 columns, 3087822 rows
4.551653447696762 % of missing values.


In [7]:
import datetime
import dateutil
from gensim.models import KeyedVectors
import nltk
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")
import langdetect

## TradeSleuth

In [30]:
df = pd.read_csv("../tradesleuth/timber.csv")

Pandas Apply: 100%|██████████| 3087822/3087822 [00:14<00:00, 205891.81it/s]


In [31]:

print(f"\nColumns dropped correctly: {list(df.columns)==list(cleaned_data.columns)}")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['Date'])/len(raw_data)} % of incorrect dates.")
print(f"Clean Dates: {100*count_diff(df, cleaned_data, ['Date'])/len(df)} % of incorrect dates.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['TradingCountryCode'])/len(raw_data)} % of misspelled TradingCountryCode.")
print(f"Clean TradingCountryCode: {100*count_diff(df, cleaned_data, ['TradingCountryCode'])/len(df)} % of misspelled TradingCountryCode.")

print(f"Add new TradingCountryName: {100*count_diff(df, cleaned_data, ['TradingCountryName'])/len(df)} % of misspelled TradingCountryName.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['DestinationCountryCode'])/len(raw_data)} % of misspelled DestinationCountryCode.")
print(f"Clean DestinationCountryCode: {100*count_diff(df, cleaned_data, ['DestinationCountryCode'])/len(df)} % of misspelled DestinationCountryCode.")

print(f"Add new DestinationCountryName: {100*count_diff(df, cleaned_data, ['DestinationCountryName'])/len(df)} % of misspelled DestinationCountryName.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['HS'])/len(raw_data)} % of incorrect HS.")
print(f"Clean HS: {100*count_diff(df, cleaned_data, ['HS'])/len(df)} % of incorrect HS.")

print(f"Add new HTS4: {100*count_diff(df, cleaned_data, ['HTS4'])/len(df)} % of incorrect HTS4.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['PortUnladingEng'])/len(raw_data)} % of misspelled PortUnladingEng.")
print(f"Clean PortUnladingEng: {100*count_diff(df, cleaned_data, ['PortUnladingEng'])/len(df)} % of misspelled PortUnladingEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['NetWeightKG'])/len(raw_data)} % of incorrect NetWeightKG.")
print(f"Clean NetWeightKG: {100*count_diff(df, cleaned_data, ['NetWeightKG'])/len(df)} % of incorrect NetWeightKG.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['StatValue_USD'])/len(raw_data)} % of incorrect StatValue_USD.")
print(f"Clean StatValue_USD: {100*count_diff(df, cleaned_data, ['StatValue_USD'])/len(df)} % of incorrect StatValue_USD.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['USD_per_KG'])/len(raw_data)} % of incorrect USD_per_KG.")
print(f"Clean USD_per_KG: {100*count_diff(df, cleaned_data, ['USD_per_KG'])/len(df)} % of incorrect USD_per_KG.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['SenderNameEng'])/len(df)} % of misspelled SenderNameEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['SenderNameEng'])/len(df)} % of misspelled SenderNameEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['SenderCompanyKeywordsEng'])/len(raw_data)} % of misspelled SenderCompanyKeywordsEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['SenderCompanyKeywordsEng'])/len(df)} % of misspelled SenderCompanyKeywordsEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['RecipientNameEng'])/len(df)} % of misspelled RecipientNameEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['RecipientNameEng'])/len(df)} % of misspelled RecipientNameEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['RecipientCompanyKeywordsEng'])/len(raw_data)} % of misspelled RecipientCompanyKeywordsEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['RecipientCompanyKeywordsEng'])/len(df)} % of misspelled RecipientCompanyKeywordsEng.")


Columns dropped correctly: True

Before cleaning: 100.0 % of incorrect dates.
Clean Dates: 0.0 % of incorrect dates.

Before cleaning: 82.4726943457233 % of misspelled TradingCountryCode.
Clean TradingCountryCode: 6.477057291514861e-05 % of misspelled TradingCountryCode.
Add new TradingCountryName: 6.477057291514861e-05 % of misspelled TradingCountryName.

Before cleaning: 80.16397965944928 % of misspelled DestinationCountryCode.
Clean DestinationCountryCode: 17.923118625361177 % of misspelled DestinationCountryCode.
Add new DestinationCountryName: 17.923118625361177 % of misspelled DestinationCountryName.

Before cleaning: 92.7261998910559 % of incorrect HS.
Clean HS: 0.0 % of incorrect HS.
Add new HTS4: 0.0 % of incorrect HTS4.

Before cleaning: 96.14647476441323 % of misspelled PortUnladingEng.
Clean PortUnladingEng: 0.0 % of misspelled PortUnladingEng.

Before cleaning: 99.94614326862106 % of incorrect NetWeightKG.
Clean NetWeightKG: 0.0 % of incorrect NetWeightKG.

Before cleanin

In [32]:
# After exiting, you will see the consumed time and iterations for each prompt, paste them here
print(f"Clean data by TradeSleuth:\n")
print(f"{len(list(df.columns))} columns, {len(df)} rows")
print(f"{100*df.isna().sum().sum()/(len(df)*len(df.columns))} % of missing values.")
print(f"{100*count_diff(df, cleaned_data, num_cols)/(len(df)*len(num_cols))} % of incorrect formats.")
print(f"{100*count_diff(df, cleaned_data, str_cols)/(len(df)*len(str_cols))} % of misspellings.")
print(f"\n{100-100*count_diff(df, cleaned_data, list(cleaned_data.columns))/(len(df)*len(cleaned_data.columns))} % of correct values.")

Time = [79.65075605700258, 59.51330720199621, 73.73047116296948, 70.54568276699865, 62.25828658399405, 130.07804629698512, 89.5422365150007, 59.6199929129798, 61.813277503999416, 42.0794862699986, 99.94885707698995, 106.93385059898719, 187.995986814989, 114.95132225798443, 158.38310202999855, 78.77653913199902, 9.798073577985633, 65.30802252300782, 82.57840017799754, 101.65643327499856, 104.15271733998088, 145.36316845499096, 100.75866600999143, 172.53604486602126]
Iteration = [1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 3, 1, 1]

total_time = np.sum(Time)
total_iteration = np.sum(Iteration)
first_time_cnt = 0
for i in Iteration:
  if Iteration[i] == 1:
    first_time_cnt += 1
print(f"\nAverage time to generate correct code: {total_time/len(Time)} s.")
print(f"Average # of revises to generate correct code: {total_iteration/len(Iteration)}.")
print(f"Accepted first-time codes: {first_time_cnt}/{len(Iteration)}.")


Clean data by TradeSleuth:

23 columns, 3087822 rows
4.416985570214639 % of missing values.
0.6251299459619111 % of incorrect formats.
2.7574128301437066 % of misspellings.

98.16966668471794 % of correct values.

Average time to generate correct code: 94.08219697532695 s.
Average # of revises to generate correct code: 1.2916666666666667.
Accepted first-time codes: 18/24.


## Baseline 1

In [14]:
df = pd.read_csv("../baseline_1/timber.csv")

In [15]:

print(f"\nColumns dropped correctly: {list(df.columns)==list(cleaned_data.columns)}")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['Date'])/len(raw_data)} % of incorrect dates.")
print(f"Clean Dates: {100*count_diff(df, cleaned_data, ['Date'])/len(df)} % of incorrect dates.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['TradingCountryCode'])/len(raw_data)} % of misspelled TradingCountryCode.")
print(f"Clean TradingCountryCode: {100*count_diff(df, cleaned_data, ['TradingCountryCode'])/len(df)} % of misspelled TradingCountryCode.")

print(f"Add new TradingCountryName: {100*count_diff(df, cleaned_data, ['TradingCountryName'])/len(df)} % of misspelled TradingCountryName.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['DestinationCountryCode'])/len(raw_data)} % of misspelled DestinationCountryCode.")
print(f"Clean DestinationCountryCode: {100*count_diff(df, cleaned_data, ['DestinationCountryCode'])/len(df)} % of misspelled DestinationCountryCode.")

print(f"Add new DestinationCountryName: {100*count_diff(df, cleaned_data, ['DestinationCountryName'])/len(df)} % of misspelled DestinationCountryName.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['HS'])/len(raw_data)} % of incorrect HS.")
print(f"Clean HS: {100*count_diff(df, cleaned_data, ['HS'])/len(df)} % of incorrect HS.")

print(f"Add new HTS4: {100*count_diff(df, cleaned_data, ['HTS4'])/len(df)} % of incorrect HTS4.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['PortUnladingEng'])/len(raw_data)} % of misspelled PortUnladingEng.")
print(f"Clean PortUnladingEng: {100*count_diff(df, cleaned_data, ['PortUnladingEng'])/len(df)} % of misspelled PortUnladingEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['NetWeightKG'])/len(raw_data)} % of incorrect NetWeightKG.")
print(f"Clean NetWeightKG: {100*count_diff(df, cleaned_data, ['NetWeightKG'])/len(df)} % of incorrect NetWeightKG.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['StatValue_USD'])/len(raw_data)} % of incorrect StatValue_USD.")
print(f"Clean StatValue_USD: {100*count_diff(df, cleaned_data, ['StatValue_USD'])/len(df)} % of incorrect StatValue_USD.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['USD_per_KG'])/len(raw_data)} % of incorrect USD_per_KG.")
print(f"Clean USD_per_KG: {100*count_diff(df, cleaned_data, ['USD_per_KG'])/len(df)} % of incorrect USD_per_KG.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['SenderNameEng'])/len(df)} % of misspelled SenderNameEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['SenderNameEng'])/len(df)} % of misspelled SenderNameEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['SenderCompanyKeywordsEng'])/len(raw_data)} % of misspelled SenderCompanyKeywordsEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['SenderCompanyKeywordsEng'])/len(df)} % of misspelled SenderCompanyKeywordsEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['RecipientNameEng'])/len(df)} % of misspelled RecipientNameEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['RecipientNameEng'])/len(df)} % of misspelled RecipientNameEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['RecipientCompanyKeywordsEng'])/len(raw_data)} % of misspelled RecipientCompanyKeywordsEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['RecipientCompanyKeywordsEng'])/len(df)} % of misspelled RecipientCompanyKeywordsEng.")


Columns dropped correctly: True

Before cleaning: 100.0 % of incorrect dates.
Clean Dates: 54.969036427617915 % of incorrect dates.

Before cleaning: 82.4726943457233 % of misspelled TradingCountryCode.
Clean TradingCountryCode: 99.6489434947999 % of misspelled TradingCountryCode.
Add new TradingCountryName: 99.6489434947999 % of misspelled TradingCountryName.

Before cleaning: 80.16397965944928 % of misspelled DestinationCountryCode.
Clean DestinationCountryCode: 64.01881326060894 % of misspelled DestinationCountryCode.
Add new DestinationCountryName: 64.45222554927065 % of misspelled DestinationCountryName.

Before cleaning: 92.7261998910559 % of incorrect HS.
Clean HS: 65.363774207192 % of incorrect HS.
Add new HTS4: 43.395409450415215 % of incorrect HTS4.

Before cleaning: 96.14647476441323 % of misspelled PortUnladingEng.
Clean PortUnladingEng: 84.0574683385247 % of misspelled PortUnladingEng.

Before cleaning: 99.94614326862106 % of incorrect NetWeightKG.
Clean NetWeightKG: 86.1

In [16]:
print(f"Clean data by baseline-1:\n")
print(f"{len(list(df.columns))} columns, {len(df)} rows")
print(f"{100*df.isna().sum().sum()/(len(df)*len(df.columns))} % of missing values.")
print(f"{100*count_diff(df, cleaned_data, num_cols)/(len(df)*len(num_cols))} % of incorrect formats.")
print(f"{100*count_diff(df, cleaned_data, str_cols)/(len(df)*len(str_cols))} % of misspellings.")
print(f"\n{100-100*count_diff(df, cleaned_data, list(cleaned_data.columns))/(len(df)*len(cleaned_data.columns))} % of correct values.")

Time = [36.93427055608481, 107.01350814756006, 33.00332809984684, 3.4735623924061656, 18.158847643993795, 5.664895256981254, 8.665143555030227, 61.78852176852524, 32.51175516564399, 4.910342332907021, 6.8554142117500305, 13.825508235022426, 81.27918144781142, 77.9169830866158, 32.45436257899928, 38.75309891100005, 56.48922213700098, 32.20490395100023, 57.67274600099881, 29.618397694999658, 83.40509776999897, 31.575577438001346, 48.39258606599833, 122.50975609499801]
Iteration = [1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 3, 1, 2, 1, 2, 3]

total_time = np.sum(Time)
total_iteration = np.sum(Iteration)
first_time_cnt = 0
for i in Iteration:
  if Iteration[i] == 1:
    first_time_cnt += 1
print(f"\nAverage time to generate correct code: {total_time/len(Time)} s.")
print(f"Average # of revises to generate correct code: {total_iteration/len(Iteration)}.")
print(f"Accepted first-time codes: {first_time_cnt}/{len(Iteration)}.")


Clean data by baseline-1:

23 columns, 3087822 rows
8.6779993766818 % of missing values.
64.85602797052421 % of incorrect formats.
74.69186770082352 % of misspellings.

29.584584355828355 % of correct values.

Average time to generate correct code: 42.71154210596561 s.
Average # of revises to generate correct code: 1.5.
Accepted first-time codes: 15/24.


## Baseline 2

In [1]:
df = pd.read_csv("../baseline_2/timber.csv")

In [35]:

print(f"\nColumns dropped correctly: {list(df.columns)==list(cleaned_data.columns)}")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['Date'])/len(raw_data)} % of incorrect dates.")
print(f"Clean Dates: {100*count_diff(df, cleaned_data, ['Date'])/len(df)} % of incorrect dates.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['TradingCountryCode'])/len(raw_data)} % of misspelled TradingCountryCode.")
print(f"Clean TradingCountryCode: {100*count_diff(df, cleaned_data, ['TradingCountryCode'])/len(df)} % of misspelled TradingCountryCode.")

print(f"Add new TradingCountryName: {100*count_diff(df, cleaned_data, ['TradingCountryName'])/len(df)} % of misspelled TradingCountryName.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['DestinationCountryCode'])/len(raw_data)} % of misspelled DestinationCountryCode.")
print(f"Clean DestinationCountryCode: {100*count_diff(df, cleaned_data, ['DestinationCountryCode'])/len(df)} % of misspelled DestinationCountryCode.")

print(f"Add new DestinationCountryName: {100*count_diff(df, cleaned_data, ['DestinationCountryName'])/len(df)} % of misspelled DestinationCountryName.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['HS'])/len(raw_data)} % of incorrect HS.")
print(f"Clean HS: {100*count_diff(df, cleaned_data, ['HS'])/len(df)} % of incorrect HS.")

print(f"Add new HTS4: {100*count_diff(df, cleaned_data, ['HTS4'])/len(df)} % of incorrect HTS4.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['PortUnladingEng'])/len(raw_data)} % of misspelled PortUnladingEng.")
print(f"Clean PortUnladingEng: {100*count_diff(df, cleaned_data, ['PortUnladingEng'])/len(df)} % of misspelled PortUnladingEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['NetWeightKG'])/len(raw_data)} % of incorrect NetWeightKG.")
print(f"Clean NetWeightKG: {100*count_diff(df, cleaned_data, ['NetWeightKG'])/len(df)} % of incorrect NetWeightKG.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['StatValue_USD'])/len(raw_data)} % of incorrect StatValue_USD.")
print(f"Clean StatValue_USD: {100*count_diff(df, cleaned_data, ['StatValue_USD'])/len(df)} % of incorrect StatValue_USD.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['USD_per_KG'])/len(raw_data)} % of incorrect USD_per_KG.")
print(f"Clean USD_per_KG: {100*count_diff(df, cleaned_data, ['USD_per_KG'])/len(df)} % of incorrect USD_per_KG.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['SenderNameEng'])/len(df)} % of misspelled SenderNameEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['SenderNameEng'])/len(df)} % of misspelled SenderNameEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['SenderCompanyKeywordsEng'])/len(raw_data)} % of misspelled SenderCompanyKeywordsEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['SenderCompanyKeywordsEng'])/len(df)} % of misspelled SenderCompanyKeywordsEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['RecipientNameEng'])/len(df)} % of misspelled RecipientNameEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['RecipientNameEng'])/len(df)} % of misspelled RecipientNameEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['RecipientCompanyKeywordsEng'])/len(raw_data)} % of misspelled RecipientCompanyKeywordsEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['RecipientCompanyKeywordsEng'])/len(df)} % of misspelled RecipientCompanyKeywordsEng.")


Columns dropped correctly: True

Before cleaning: 100.0 % of incorrect dates.
Clean Dates: 0.0 % of incorrect dates.

Before cleaning: 82.4726943457233 % of misspelled TradingCountryCode.
Clean TradingCountryCode: 18.65680729005752 % of misspelled TradingCountryCode.
Add new TradingCountryName: 18.65680729005752 % of misspelled TradingCountryName.

Before cleaning: 80.16397965944928 % of misspelled DestinationCountryCode.
Clean DestinationCountryCode: 0.8124172960747089 % of misspelled DestinationCountryCode.
Add new DestinationCountryName: 0.8124172960747089 % of misspelled DestinationCountryName.

Before cleaning: 92.7261998910559 % of incorrect HS.
Clean HS: 0.0 % of incorrect HS.
Add new HTS4: 0.0 % of incorrect HTS4.

Before cleaning: 96.14647476441323 % of misspelled PortUnladingEng.
Clean PortUnladingEng: 0.0 % of misspelled PortUnladingEng.

Before cleaning: 99.94614326862106 % of incorrect NetWeightKG.
Clean NetWeightKG: 0.0 % of incorrect NetWeightKG.

Before cleaning: 99.99

In [36]:
print(f"Clean data by baseline-2:\n")
print(f"{len(list(df.columns))} columns, {len(df)} rows")
print(f"{100*df.isna().sum().sum()/(len(df)*len(df.columns))} % of missing values.")
print(f"{100*count_diff(df, cleaned_data, num_cols)/(len(df)*len(num_cols))} % of incorrect formats.")
print(f"{100*count_diff(df, cleaned_data, str_cols)/(len(df)*len(str_cols))} % of misspellings.")
print(f"\n{100-100*count_diff(df, cleaned_data, list(cleaned_data.columns))/(len(df)*len(cleaned_data.columns))} % of correct values.")

Time = [70.31692368071526, 186.8816757388413, 168.90672597661614, 167.44252010062337, 79.34907004702836, 203.12355579063296, 296.4537397371605, 227.95822145789862, 142.40654882136732, 137.60550732538104, 179.05404731351882, 71.52330657374114, 91.02093854639679, 154.09032074455172, 75.03718110453337, 339.73870580643415, 91.82672706991434, 87.93194920290262, 71.70418223086745, 54.12616850901395, 69.1190603254363, 101.66071887407452, 75.96594125032425, 88.29510616976768]
Iteration = [1, 2, 2, 2, 1, 3, 3, 3, 2, 2, 2, 1, 1, 3, 1, 2, 4, 4, 2, 2, 2, 2, 2, 2]

applied = [0, 2, 3, 7, 8, 9, 10, 11, 14, 16, 17, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]

total_time = np.sum(Time)
total_iteration = np.sum(Iteration)
first_time_cnt = 0
for i in Iteration:
  if Iteration[i] == 1:
    first_time_cnt += 1
print(f"\nAverage time to generate correct code: {total_time/len(Time)} s.")
print(f"Average # of revises to generate correct code: {total_iteration/len(Iteration)}.")
print(f"Accepted first-time codes: {first_time_cnt}/{len(Iteration)}.")


Clean data by baseline-2:

23 columns, 3087822 rows
4.75665794319694 % of missing values.
0.6251299459619111 % of incorrect formats.
4.041823255754629 % of misspellings.

97.44369557459004 % of correct values.

Average time to generate correct code: 134.64745176657257 s.
Average # of revises to generate correct code: 2.125.
Accepted first-time codes: 5/24.


## Baseline 3

In [42]:
df = pd.read_csv("../baseline_3/timber.csv")

In [46]:

print(f"\nColumns dropped correctly: {list(df.columns)==list(cleaned_data.columns)}")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['Date'])/len(raw_data)} % of incorrect dates.")
print(f"Clean Dates: {100*count_diff(df, cleaned_data, ['Date'])/len(df)} % of incorrect dates.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['TradingCountryCode'])/len(raw_data)} % of misspelled TradingCountryCode.")
print(f"Clean TradingCountryCode: {100*count_diff(df, cleaned_data, ['TradingCountryCode'])/len(df)} % of misspelled TradingCountryCode.")

print(f"Add new TradingCountryName: {100*count_diff(df, cleaned_data, ['TradingCountryName'])/len(df)} % of misspelled TradingCountryName.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['DestinationCountryCode'])/len(raw_data)} % of misspelled DestinationCountryCode.")
print(f"Clean DestinationCountryCode: {100*count_diff(df, cleaned_data, ['DestinationCountryCode'])/len(df)} % of misspelled DestinationCountryCode.")

print(f"Add new DestinationCountryName: {100*count_diff(df, cleaned_data, ['DestinationCountryName'])/len(df)} % of misspelled DestinationCountryName.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['HS'])/len(raw_data)} % of incorrect HS.")
print(f"Clean HS: {100*count_diff(df, cleaned_data, ['HS'])/len(df)} % of incorrect HS.")

print(f"Add new HTS4: {100*count_diff(df, cleaned_data, ['HTS4'])/len(df)} % of incorrect HTS4.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['PortUnladingEng'])/len(raw_data)} % of misspelled PortUnladingEng.")
print(f"Clean PortUnladingEng: {100*count_diff(df, cleaned_data, ['PortUnladingEng'])/len(df)} % of misspelled PortUnladingEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['NetWeightKG'])/len(raw_data)} % of incorrect NetWeightKG.")
print(f"Clean NetWeightKG: {100*count_diff(df, cleaned_data, ['NetWeightKG'])/len(df)} % of incorrect NetWeightKG.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['StatValue_USD'])/len(raw_data)} % of incorrect StatValue_USD.")
print(f"Clean StatValue_USD: {100*count_diff(df, cleaned_data, ['StatValue_USD'])/len(df)} % of incorrect StatValue_USD.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['USD_per_KG'])/len(raw_data)} % of incorrect USD_per_KG.")
print(f"Clean USD_per_KG: {100*count_diff(df, cleaned_data, ['USD_per_KG'])/len(df)} % of incorrect USD_per_KG.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['SenderNameEng'])/len(df)} % of misspelled SenderNameEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['SenderNameEng'])/len(df)} % of misspelled SenderNameEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['SenderCompanyKeywordsEng'])/len(raw_data)} % of misspelled SenderCompanyKeywordsEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['SenderCompanyKeywordsEng'])/len(df)} % of misspelled SenderCompanyKeywordsEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['RecipientNameEng'])/len(df)} % of misspelled RecipientNameEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['RecipientNameEng'])/len(df)} % of misspelled RecipientNameEng.")

print(f"\nBefore cleaning: {100*count_diff(raw_data, cleaned_data, ['RecipientCompanyKeywordsEng'])/len(raw_data)} % of misspelled RecipientCompanyKeywordsEng.")
print(f"Clean strings: {100*count_diff(df, cleaned_data, ['RecipientCompanyKeywordsEng'])/len(df)} % of misspelled RecipientCompanyKeywordsEng.")


Columns dropped correctly: True

Before cleaning: 100.0 % of incorrect dates.
Clean Dates: 0.0 % of incorrect dates.

Before cleaning: 82.4726943457233 % of misspelled TradingCountryCode.
Clean TradingCountryCode: 6.477057291514861e-05 % of misspelled TradingCountryCode.
Add new TradingCountryName: 6.477057291514861e-05 % of misspelled TradingCountryName.

Before cleaning: 80.16397965944928 % of misspelled DestinationCountryCode.
Clean DestinationCountryCode: 0.22663223463010498 % of misspelled DestinationCountryCode.
Add new DestinationCountryName: 0.22663223463010498 % of misspelled DestinationCountryName.

Before cleaning: 92.7261998910559 % of incorrect HS.
Clean HS: 0.0 % of incorrect HS.
Add new HTS4: 0.0 % of incorrect HTS4.

Before cleaning: 96.14647476441323 % of misspelled PortUnladingEng.
Clean PortUnladingEng: 0.0 % of misspelled PortUnladingEng.

Before cleaning: 99.94614326862106 % of incorrect NetWeightKG.
Clean NetWeightKG: 91.5089665142615 % of incorrect NetWeightKG.


In [3]:
print(f"Clean data by baseline-3:\n")
print(f"{len(list(df.columns))} columns, {len(df)} rows")
print(f"{100*df.isna().sum().sum()/(len(df)*len(df.columns))} % of missing values.")
print(f"{100*count_diff(df, cleaned_data, num_cols)/(len(df)*len(num_cols))} % of incorrect formats.")
print(f"{100*count_diff(df, cleaned_data, str_cols)/(len(df)*len(str_cols))} % of misspellings.")
print(f"\n{100-100*count_diff(df, cleaned_data, list(cleaned_data.columns))/(len(df)*len(cleaned_data.columns))} % of correct values.")

Time = [405.00571952201426, 364.8505232851021, 90.49188285390846, 230.93646942998748, 396.01469031802844, 239.7160882720491, 51.28350269200746, 647.606322239968, 362.9263410329586, 96.21659857500345, 981.0702107369434, 244.83681565499865, 203.46100543194916, 385.66700267698616, 58.433924220036715, 422.17109180195257, 365.48042021796573, 17.680476023000665, 22.339896818972193, 22.754380203899927, 170.66795389191248, 233.98927278094925, 392.30982334379967, 58.13455335004255]
Iteration = [2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 5, 1]

total_time = np.sum(Time)
total_iteration = np.sum(Iteration)
first_time_cnt = 0
for i in Iteration:
  if Iteration[i] == 1:
    first_time_cnt += 1
print(f"\nAverage time to generate correct code: {total_time/len(Time)} s.")
print(f"Average # of revises to generate correct code: {total_iteration/len(Iteration)}.")
print(f"Accepted first-time codes: {first_time_cnt}/{len(Iteration)}.")


Clean data by baseline-3:

23 columns, 3087822 rows
8.237331375797654 % of missing values.
12.806612557330054 % of incorrect formats.
6.832627807411976 % of misspellings.

90.5699875187106 % of correct values.

Average time to generate correct code: 269.3352068906015 s.
Average # of revises to generate correct code: 1.4166666666666667.
Accepted first-time codes: 17/24.
